In [ ]:
import os
from pandas import read_table
from benbiohelpers.FileSystemHandling.DirectoryHandling import getFilesInDirectory
from bpde_chromatin_analysis.helper_scripts.BPDE_DataDir import getDataDir
from mutperiodpy.helper_scripts.UsefulFileSystemFunctions import DataTypeStr, Metadata
from mutperiodpy.quick_scripts.GetNRL import getNRL
from mutperiodpy.GeneratePlotnineFigures import parseAndPlotPeriodicity, parseNucleosomeCountsDataForPlotting, plotPeriodicity

alexandrovDataDirectory = os.path.join(getDataDir(), "Alexandrov_LUAD")

In [ ]:
SINGLE_NUC = 1
NUC_GROUP = 2

alexandrovNormalizedCounts = {SINGLE_NUC:dict(), NUC_GROUP:dict()}
NRLs = dict()

nucleosomeMapNameWhitelist = ['hg19_hybrid_nuc_map', 'hg19_LCL_MNase_nuc_map', 'hg19_NHF1_MNase_nuc_map']
# nucleosomeMapNameWhitelist += ['hg19_LCL_MNase_nuc_map_euchromatin', 'hg19_LCL_MNase_nuc_map_heterochromatin',
#                               'hg19_LCL_MNase_nuc_map_stringent_euchromatin']

for nucleosomeCountsFilePath in getFilesInDirectory(alexandrovDataDirectory, DataTypeStr.normNucCounts+".tsv"):

    countsMetadata = Metadata(nucleosomeCountsFilePath)
    if countsMetadata.nucPosName not in nucleosomeMapNameWhitelist: continue

    if "nuc-group" in nucleosomeCountsFilePath: radius = NUC_GROUP
    else: radius = SINGLE_NUC

    alexandrovNormalizedCounts[radius][countsMetadata.nucPosName] = read_table(nucleosomeCountsFilePath)

    if countsMetadata.nucPosName not in NRLs: NRLs[countsMetadata.nucPosName] = getNRL([countsMetadata.baseNucPosFilePath])[0]

In [ ]:
for radius in alexandrovNormalizedCounts:
    for nucleosomeMapName in alexandrovNormalizedCounts[radius]:
        plot = parseAndPlotPeriodicity(alexandrovNormalizedCounts[radius][nucleosomeMapName], dataCol = "Normalized_Aligned_Strands",
                                       smoothTranslational = True, nucRepLen = NRLs[nucleosomeMapName],
                                       title = f"Alexandrov {nucleosomeMapName} Trinuc Normalized", yAxisLabel = "Normalized Counts")
        print(plot)